In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd

import boto3

In [43]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.rottentomatoes.com/browse/movies_in_theaters/sort:popular")

max_load_time = 120  # Maximum allowed time in seconds (2 minutes) after button press

while True:
    try:
        # Wait for the button to be present and clickable
        load_more_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-qa='dlp-load-more-button']"))
        )
        
        # Click the button
        load_more_button.click()
        print("Clicked 'Load More' button")
        
        # Start timing after button press
        start_time = time.time()
        
        # Wait for new content to load, but stop if it takes too long
        while time.time() - start_time < max_load_time:
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "//button[@data-qa='dlp-load-more-button']"))
                )
                break  # Exit the loop if new content is detected
            except:
                pass  # Keep waiting until max_load_time is exceeded
        else:
            print("Load process exceeded 2 minutes after button press. Stopping.")
            break
        
        # Optional: Wait a bit before checking again (to allow new content to load)
        time.sleep(2)
    
    except:
        # If the button is no longer found, break out of the loop
        print("No more 'Load More' button found. Exiting loop.")
        break

print("All content loaded or time limit reached.")

html = driver.page_source
time.sleep(2)
print("saved to a file")

elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-qa^="discovery-media-list-item"]')

tiles = driver.find_elements(By.CSS_SELECTOR, "[data-qa*='discovery-media-list-item-caption']")


concluding_list = []
for element in elements:
    concluding_list.append(element.get_attribute("href"))

elements2 = driver.find_elements(By.CSS_SELECTOR, "span[data-qa='discovery-media-list-item-title']")

movie_name_list =[]

for i in elements2:
    movie_name_list.append(i.text)

audience_score_list = []

for i in tiles:
    try:
        score_element = i.find_element(By.CSS_SELECTOR, 'rt-text[slot="audienceScore"]')
        score_text = score_element.text.strip()
        audience_score_list.append(score_text if score_text else None)
    except:
        audience_score_list.append(None)

critic_score_list = []

for i in tiles:
    try:
        score_element = i.find_element(By.CSS_SELECTOR, 'rt-text[slot="criticsScore"]')
        score_text = score_element.text.strip()
        critic_score_list.append(score_text if score_text else None)
    except:
        critic_score_list.append(None)


driver.quit()

concluding_list


['https://www.rottentomatoes.com/m/anora',
 'https://www.rottentomatoes.com/m/captain_america_brave_new_world',
 'https://www.rottentomatoes.com/m/last_breath_2025',
 'https://www.rottentomatoes.com/m/companion_2025',
 'https://www.rottentomatoes.com/m/the_monkey',
 'https://www.rottentomatoes.com/m/mickey_17',
 'https://www.rottentomatoes.com/m/a_complete_unknown',
 'https://www.rottentomatoes.com/m/flow_2024',
 'https://www.rottentomatoes.com/m/conclave',
 'https://www.rottentomatoes.com/m/the_substance',
 'https://www.rottentomatoes.com/m/im_still_here_2024',
 'https://www.rottentomatoes.com/m/a_real_pain',
 'https://www.rottentomatoes.com/m/love_hurts_2025',
 'https://www.rottentomatoes.com/m/one_of_them_days',
 'https://www.rottentomatoes.com/m/riff_raff_2025',
 'https://www.rottentomatoes.com/m/my_dead_friend_zoe',
 'https://www.rottentomatoes.com/m/nickel_boys',
 'https://www.rottentomatoes.com/m/paddington_in_peru',
 'https://www.rottentomatoes.com/m/wicked_2024',
 'https://www

In [38]:
movie_name_list

['Anora',
 'Captain America: Brave New World',
 'Last Breath',
 'Companion',
 'The Monkey',
 'Mickey 17',
 'A Complete Unknown',
 'Flow',
 'Conclave',
 'The Substance',
 "I'm Still Here",
 'A Real Pain',
 'Love Hurts',
 'One of Them Days',
 'Riff Raff',
 'My Dead Friend Zoe',
 'Nickel Boys',
 'Paddington in Peru',
 'Wicked',
 'The Unbreakable Boy',
 'Mufasa: The Lion King',
 'Heart Eyes',
 'Flight Risk',
 'Sonic the Hedgehog 3',
 'The Last Showgirl',
 'Becoming Led Zeppelin',
 'Cleaner',
 'The Klezmer Project']

In [39]:
# concluding_list = ['https://www.rottentomatoes.com/m/emilia_perez',
#  'https://www.rottentomatoes.com/m/trial_by_fire',
#  'https://www.rottentomatoes.com/m/venom_the_last_dance']

# movie_name_list = ["Emilia Perez", "Trial by Fire", "Venom the Last Dance"]

g = 0

final_list = []

driver = webdriver.Chrome()

for i in concluding_list:
    
    
    #add in previous lists to dict
    movie_dict = {'Movie Title': movie_name_list[g],
                  'Critic Score': critic_score_list[g],
                  'Audience Score': audience_score_list[g]}
    
    #go to the website for the specific movie
    driver.get(i)

    #grab the synopsis
    try:
        synopsis_element = driver.find_element(By.CSS_SELECTOR, 'rt-text[data-qa="synopsis-value"]')
        synopsis = synopsis_element.text.strip() if synopsis_element else None
    except:
        synopsis = None

    movie_dict['Synopsis'] = synopsis

    #grab the cast list
    cast_list = driver.find_elements(By.XPATH, "//div[p[@data-qa='person-role'][normalize-space() != 'Director']]/p[@data-qa='person-name']")

    cast_temp = ', '.join(i.text for i in cast_list)
    
    movie_dict['Cast'] = cast_temp

    #grab all other data from large "details" section on the bottom of the site
    x = driver.find_elements(By.CLASS_NAME, 'category-wrap')

    for e in x:
        cleaned_text = e.text.split()
        if len(cleaned_text) > 1:
            if cleaned_text[0] in ['Original', 'Production', 'Sound']:
                key = cleaned_text[0] + ' ' + cleaned_text[1]
                value = ' '.join(cleaned_text[2:])
            elif cleaned_text[0] in ['Aspect', 'Release', 'Rerelease']:
                key = cleaned_text[0] + ' ' + cleaned_text[1] + ' ' + cleaned_text[2]
                value = ' '.join(cleaned_text[3:])
            elif cleaned_text[0] == 'Box':
                key = cleaned_text[0] + ' ' + cleaned_text[1] + ' ' + cleaned_text[2] + ' ' + cleaned_text[3]
                value = ' '.join(cleaned_text[4:])
            else:
                key = cleaned_text[0]
                value = ' '.join(cleaned_text[1:])
        else:
            key = cleaned_text[0]
            value = ""

        movie_dict[key] = value  
    
    g += 1

    final_list.append(movie_dict)  


#convert to DF
final_df = pd.DataFrame(final_list)

#write to CSV
final_df.to_csv("rotten_tomatoes_data.csv")

driver.quit()

In [40]:
bucket_name = 'practice-bucket-24'
file_key = 'rotten_tomatoes_data.csv'

s3_client = boto3.client('s3')

In [41]:
### Getting Object and Updating ###

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
temp_df = pd.read_csv(response.get("Body"))

combined_df = pd.concat([final_df, temp_df])
combined_df = combined_df.drop_duplicates(subset=['Movie Title', 'Director'], keep='last')


combined_df.to_csv("rotten_tomatoes_data_update.csv", index = False)

s3_client.upload_file(Bucket = bucket_name, Key = file_key, Filename= "rotten_tomatoes_data_update.csv")